# Use the JPL Horizons API and plot Distance to Mars

In [ ]:
import requests
import datetime as dt 
import matplotlib.pyplot as plt
import matplotlib.dates as md
import numpy as np
from scipy.signal import argrelextrema

In [ ]:
# Get the data
print("PLEASE WAIT... May take a minute")
BASE_URL = 'https://ssd.jpl.nasa.gov/horizons_batch.cgi'
target='499' #mars
quantities = '"4,20"' #Must have internal quotes!
START_TIME = '2015-OCT-1'
STOP_TIME = '2025-OCT-7'
SITE_COORD = '"238.5056,8.5816,100.0"' #Must have internal quotes!
params = {
    "batch":"1",
    "COMMAND":'499',
    "CENTER":'coord@399',
    "COORD_TYPE":'GEODETIC',
    "SITE_COORD":SITE_COORD,
    "MAKE_EPHEM":'YES',
    "TABLE_TYPE":'OBSERVER',
    "START_TIME":START_TIME,
    "STOP_TIME":STOP_TIME,
    "STEP_SIZE":'9m',
    "CAL_FORMAT":'CAL',
    "TIME_DIGITS":'MINUTES',
    "ANG_FORMAT":'HMS',
    "OUT_UNITS":'KM-S',
    "RANGE_UNITS":'AU',
    "APPARENT":'AIRLESS',
    "SUPPRESS_RANGE_RATE":'NO',
    "SKIP_DAYLT":'NO',
    "EXTRA_PREC":'NO',
    "R_T_S_ONLY":'TVH',
    "REF_SYSTEM":'J2000',
    "CSV_FORMAT":'NO',
    "OBJ_DATA":'YES',
    "QUANTITIES":'"4,20"',
    'CSV_FORMAT':'YES'
    
}
result = requests.get(BASE_URL, params=params)
#dir(result)
#result.url
result_text = result.text
#print(result_text)
print("FINISHED")


In [ ]:
# Get lines between $$SOE and $$EOE and parse out the datatime and distance
dates = []
dist = []
parsing = False
for line in result_text.splitlines():
    if line.startswith('$$SOE'):
        parsing = True
    elif line.startswith('$$EOE'):
        parsing = False
    elif parsing == True:
        #print(line)
        parts = line.split(',')
        date_time_string = parts[0].strip() + "-00:00"
        marker = parts[2].strip()
        distance = float(parts[5].strip())
        
        if marker == 't':
            #print(date_time_string + "," + marker + "," + distance)
            # See https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes
            our_datetime = dt.datetime.strptime(date_time_string, '%Y-%b-%d %H:%M%z')
            #print(our_datetime)
            timestamp = int(dt.datetime.timestamp(our_datetime))
            #print(timestamp)
            dates.append(timestamp)
            dist.append(distance)
            #print(str(timestamp) + "," + str(distance))
print("FINISHED")        

In [ ]:
# Plot

plt.style.use('seaborn-talk')

# Distance is in AU.
# 1 au is 149597870.700 km
# Convert the disances from AU to km x 10**8
dist2 = [(149597870.7 * d)/1e8 for d in dist]
# Convert the timestamps to datetime instances
dates2 = [dt.datetime.fromtimestamp(e) for e in dates]

# Start the figure with one axis
fig, ax = plt.subplots(figsize=(16, 5))

# Create a line plot
ax.plot(dates2, dist2, 'tab:orange')

# Format the datetime for the x axis
myFmt = md.DateFormatter('%Y-%m-%d')
ax.xaxis.set_major_formatter(myFmt)

# Add the title and labels
ax.set(xlabel='Date', ylabel='km from Earth (x $10^8$)')
ax.set_facecolor((0, 0, 0.3))
ax.set_title("Distance From Earth To Mars 2016 - 2026", fontsize=26)
ttl = ax.title
ttl.set_position([.5, 1.05])

# Add text to the right
plt.text(1.16,0.88,'Closest Approaches',horizontalalignment='center',
     verticalalignment='center', transform = ax.transAxes, fontsize=24, color='black')

## Calculate the minima, the closest approaches. Add the text.
mins = argrelextrema(np.array(dist), np.less)
mins[0].tolist()
x = 1.16
y = 0.72
power = "$10^7$"
for minima_index in mins[0].tolist():
    text = f"{dt.datetime.strftime(dates2[minima_index], '%Y-%b-%d')} ({dist2[minima_index]*10:.1f}x{power}km)"
    plt.text(x,y,text,horizontalalignment='center',
             verticalalignment='center', transform = ax.transAxes, fontsize=20, color='black')
    y -= 0.1

# Plot!
plt.plot()